In [59]:
from dotenv import load_dotenv      # loads environment variables from a .env file into Python environment
import os                           # access environment variables once they've been loaded into the Python environmen

In [60]:
# print(os.getenv('OPENAI_API_KEY'))

In [61]:
from langchain_core.prompts import ChatPromptTemplate             

#  Creating a template for the prompt
template = """
Think You are data analyst in MySQL. Based on the schema below wirte Sql queries to answer the following question:
{schema}   

Question: {question}

Sql Query:
"""

prompt = ChatPromptTemplate.from_template(template)

In [62]:
# prompt.format(schema= "myshema", question="my question")

In [63]:
from sqlalchemy import create_engine
from langchain_community.utilities import SQLDatabase

# Define the connection URI
mysql_uri = "mysql+mysqlconnector://root:@localhost:3306/de"

# Create an SQLAlchemy engine
engine = create_engine(mysql_uri)

# Pass the engine to SQLDatabase
db = SQLDatabase(engine)  # db is an instance of SQLDatabase


In [64]:
# db.run("SQL Query") # runs the SQL query and returns the result
db.run("show tables") # returns a list of tables in the database

"[('employee',), ('sales',), ('users',)]"

In [65]:
# Get the schema of the database
# Schema is a list of dictionaries, where each dictionary represents a column in a table

def get_schema(_):    # For the RunnablePassthrough towork function should be consist with at least one parameter
    schema = db.get_table_info()
    return schema

In [66]:
# get_schema(None)

In [67]:
from langchain_core.output_parsers import StrOutputParser  # Get the Out Put as a String
from langchain_core.runnables import RunnablePassthrough   
from langchain_openai import ChatOpenAI                    # Importing the OpenAI Chat Model

llm = ChatOpenAI(
    temperature=0.5,
    max_tokens=100,
    model = "gpt-3.5-turbo"

)

sql_chain = (
    RunnablePassthrough.assign(schema=get_schema) 
    | prompt
    | llm
    | StrOutputParser()
)


In [68]:
user_question = 'Give me the customer name, city ,country and which is having highest top10 credit limit '
sql_chain.invoke({"question": user_question})

'Since there is no table provided in the schema that contains information about customer name, city, country, and credit limit, it is not possible to write a SQL query to answer this question. Additional information or tables containing the required data would be needed to write the query.'

In [25]:
template = """Think you are Data Analyst and use Mysql databases. Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)


In [26]:
def run_query(query):
    return db.run(query)


In [27]:
full_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema = get_schema,
        response = lambda vars: run_query(vars["query"])
    )
    | prompt_response
    | llm
    | StrOutputParser()
)


In [28]:
user_question = 'how many customers are there?'
full_chain.invoke({"question": user_question})

'There are a total of 5 customers in the database.'